# Supply Chain Advisor

<table align="left" width="100%">
  <td>
    <a href="https://colab.research.google.com/github/guruvittal/supplychaingraph/blob/main/SupplyChainAdvisor.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/guruvittal/supplychaingraph/blob/main/SupplyChainAdvisor.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/guruvittal/supplychaingraph/blob/main/SupplyChainAdvisor.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Dependencies, Libraries & Authentication

In [ ]:
#@title Install Dependencies
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('neo4j', 'neo4j'),
    ('gradio','gradio'),
    ('langchain-google-vertexai','langchain-google-vertexai'),
    ('langchain','langchain'),
    ('typing_extensions', 'typing_extensions')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U --user



In [ ]:
#@title Restart Kernel

if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
#@title Import relevant libraries

import os

import gradio as gr
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_google_vertexai import VertexAI
from langchain.prompts.prompt import PromptTemplate


from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai

In [ ]:
#@title Setting up the Auth

import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()



## Helper Functions

In [ ]:
#@title Helper Function to Run Query on Database

import pandas as pd
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
#@title Helper Function to Execute Cypher file
import urllib.request
def ExecuteCypher(CypherFile):
  CommandFile = urllib.request.urlopen(CypherFile)
  FileContent = CommandFile.read()
  response = run_query(FileContent.decode())
  return response

In [ ]:
#@title Setup Database (Graph)

from neo4j import GraphDatabase
from google.colab import userdata


# You will need to change these variables
connectionUrl = userdata.get("neo4jurl") #@param
username = userdata.get('neo4jusername') #@param
password = userdata.get('neo4jpassword') #@param

In [ ]:
#@title Verify Database Connectivity

driver = GraphDatabase.driver(connectionUrl, auth=(username, password))
driver.verify_connectivity()

In [ ]:
#@title Quick Test on the LLM

vertexai.init(project="argolis-project-340214")
# %%

llm = VertexAI(model_name = "gemini-pro", temperature=1, max_output_tokens=8192)
print(llm.invoke("What is the future of supply chain in an LLM world?"))



# Build your Supply Chain Model

In [ ]:
#@title Clean up the database

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/dropDatabase.cyp")


In [ ]:
#@title Create Locations

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createLocations.cyp")


In [ ]:
#@title Create Items

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createItems.cyp")


In [ ]:
#@title Create Item Location

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createItemLocation.cyp")


In [ ]:
#@title Create MADE_AT

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createMadeAt.cyp")


In [ ]:
#@title Create MAKES

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createMakes.cyp")

In [ ]:
#@title Create CONTAINS

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/createContains.cyp")


In [ ]:
#@title Set Forecasts

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/setForecasts.cyp")


In [ ]:
#@title Generate Supply Plan

ExecuteCypher("https://raw.githubusercontent.com/guruvittal/SupplyChainGraph/main/generateSupplyPlan.cyp")

## Run Sample Queries

In [ ]:
run_query("match (n:Location) return n.locationKey, n.address")

In [ ]:
run_query("match (n) return labels(n) as label, properties(n) as properties")

In [ ]:
graph = Neo4jGraph(
    url=connectionUrl, username=username, password=password
)

In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

# Build Text to Cypher templates

In [ ]:
from langchain_google_vertexai import ChatVertexAI

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# Hello bot
MATCH (il:ItemLocation)
return "No. of ItemLocation nodes in this model: ", count(*)," - Welcome."

# What are all the parent items and where are they produced?
MATCH (il:ItemLocation)-[:CONTAINS]->(:ItemLocation) where NOT (:ItemLocation)-[:CONTAINS]->(il)
MATCH (il)-[:MADE_AT]-(l:Location)
RETURN distinct il.itemNumber AS parentItem, l.locationDescription AS productionLocation

# What products does the Seattle supplier make?
MATCH (il)-[:MADE_AT]-(l:Location)
WHERE l.locationDescription contains 'Seattle'
RETURN il.itemNumber

# Machine plants in WV
MATCH (l:Location)-[:MAKES]->(i:Item)
WHERE l.address contains 'KY' and l.locationType contains 'PLT' and i.itemDescription contains 'Machine'
RETURN l.locationId + ' is a plant and is in' + l.address + ' and it makes:' + i.itemDescription

# Suppliers in MA
MATCH (l:Location)
WHERE l.address contains 'MA' and l.locationType contains 'SUP'
RETURN l.locationId + 'supplier is in' + l.address


# Name all suppliers for Transmission?
MATCH (il)-[:MADE_AT]->(l:Location)
WHERE il.itemNumber contains 'Transmission'
RETURN l.locationDescription

# What is the forecast for Machine in WV plant in month 2
MATCH (a:ItemLocation)-[r:HAS_FORECAST_IN]->(m:Month)
WHERE m.monthKey = "2" and a.itemLocationDescription contains 'WV' and a.locationType = 'PLT' and a.itemLocationDescription contains 'Machine'
RETURN a.itemLocationDescription,  r.forecast

# What is the forecast for Engine for supplier Cummins in month 8
MATCH (j:ItemLocation)-[q:HAS_FORECAST_IN]->(m:Month)
WHERE m.monthKey = "8" and j.itemLocationDescription contains 'Cummins' and j.locationType = 'SUP' and j.itemLocationDescription contains 'Engine'
RETURN j.itemLocationDescription,  q.forecast

# Name all suppliers for Engines?
MATCH (il)-[:MADE_AT]->(l:Location)
WHERE il.itemNumber contains 'Engine'
RETURN l.locationDescription

# Name all the suppliers for the WV plant
MATCH (plant:Location)<-[:MADE_AT]-(parentItemLocation:ItemLocation)-[:CONTAINS]->(childItemLocation:ItemLocation)-[:MADE_AT]->(supp:Location)
WHERE plant.locationDescription contains 'WV'
RETURN supp.locationId

# Name all the suppliers of Engines for the WV plant
MATCH (parentItemLocation:ItemLocation)-[:CONTAINS]->(childItemLocation:ItemLocation)-[:MADE_AT]->(supp:Location)
WHERE parentItemLocation.itemLocationDescription contains 'WV' and parentItemLocation.locationType contains 'PLT' and childItemLocation.itemNumber contains 'Engine' and supp.locationType contains 'SUP'
RETURN  supp.locationId + ' is a supplier and is in:' + supp.address + ' supplying to: ' + parentItemLocation.itemLocationDescription

The question is:
{question}"""


CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatVertexAI(temperature=0), graph=graph, verbose=True, cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [ ]:
chain.invoke("What are all the parent items and where are they produced?")

In [ ]:
chain.run("What all locations are there?")

In [ ]:
chain.run("What all itemlocations are there for the itemgoverning system SAP2 and print the list of item Location Descriptions?")

In [ ]:
chain.run("What all top level items are being produced and where")

In [ ]:
chain.run("What item does the Seattle supplier make")

In [ ]:
chain.run("Name all the suppliers of Engine for WV plant?")

# Build the Bot UI

In [ ]:

def chatbot(inputtext):
    return chain.run(inputtext)


iface = gr.Interface(fn=chatbot,
                      examples=[
                               "Name all the suppliers for the WV plant",
                               "Name all machine plants in KY",
                               "Name all suppliers in KY",
                               "Name all the suppliers for Engines",
                               "Name all the suppliers of Engines for the WV plant",
                               "What all top level items are being produced and where",
                               "What is the forecast of machine in WV for month 6",
                               "What is the forecast for Engine for supplier Cummins in month 8"
                               ],
                      title="Automotive Supply Chain Analyst",
                      inputs=gr.Textbox(),
                      outputs=[gr.Textbox()])

iface.launch(debug=True)

